
# Цель работы:
----

<br>

Провести **A/B тестирование** по реальным данным, предоставленным отделем (`ab_test_ensembles.csv`). Есть два варианта работы сервиса в отеле - `standard` и `svm_ensemble_v_1_22`, в которых работают несколько моделей классификации для целей сервиса.

<br>
------
Проведя исследование данных и тестирование необходимо ответить на **2 главных вопроса:**

1. Стоит ли нам оставить старый вариант работы сервиса или заменить его на вариант работы с моделями классификации (используем всю выборку 200к+ пользователей).
2. Сделать такой же вывод для типа пользователей (`user_type`). Стоит ли для новых (старых) пользователей оставить старый (новый) вариант работы сервиса.

In [1]:
import pandas as pd
import numpy as np
# import statistics
from scipy.stats import stats, norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
# import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
np.random.seed(42)

**Разведочный анализ данных**

In [3]:
data = pd.read_csv('./ab_test_ensembles.csv', index_col=False)
data.head()

,user_id,timestamp,group,variants,converted,location,age_group,user_type
0,9109b0dc-d393-497f-8d63-ba9a25dd16b4,2022-05-21 22:11:48.556739,control,standard,0,United Kingdom,18-25,registered_user
1,2430c3d2-f75b-4b31-8271-51b6a76c2652,2022-05-12 08:01:45.159739,control,standard,0,United Kingdom,42-49,registered_user
2,44788c4e-8dd2-4fad-b986-75e76f4adb64,2022-05-11 16:55:06.154213,treatment,svm_ensemble_v_1_22,0,United Kingdom,26-33,new_user
3,4699a417-506d-41b8-a354-6af6ad576963,2022-05-08 18:28:03.143765,treatment,svm_ensemble_v_1_22,0,United Kingdom,42-49,registered_user
4,304b0d28-bcdf-401a-9dff-66230d3ba0bc,2022-05-21 01:52:26.210827,control,standard,1,United Kingdom,42-49,registered_user


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    294478 non-null  object
 1   timestamp  294478 non-null  object
 2   group      294478 non-null  object
 3   variants   294478 non-null  object
 4   converted  294478 non-null  int64 
 5   location   294478 non-null  object
 6   age_group  294478 non-null  object
 7   user_type  294478 non-null  object
dtypes: int64(1), object(7)
memory usage: 18.0+ MB


In [5]:
data.describe(include=np.object) # type: ignore

,user_id,timestamp,group,variants,location,age_group,user_type
count,294478,294478,294478,294478,294478,294478,294478
unique,294478,294478,2,2,1,6,2
top,9109b0dc-d393-497f-8d63-ba9a25dd16b4,2022-05-21 22:11:48.556739,treatment,standard,United Kingdom,26-33,registered_user
freq,1,1,147276,147239,294478,49270,147240


Необходимо посмотреть на наличие несоответствий в данных предиктов 'group' и 'variants'. По-идее, если данные заполнены правильно, то строк с данными фильтра (data['group']== 'control') & (data['variants']== 'svm_ensemble_v_1_22') и (data['group']== 'treatment') & (data['variants']== 'standard') не должно быть.

In [10]:
len(data[(data['group']== 'control') & (data['variants']== 'svm_ensemble_v_1_22')])

1928

In [11]:
len(data[(data['group']== 'treatment') & (data['variants']== 'standard')])

1965

Есть ошибки в заполнении данных и не понятно, какое именно это должно быть сочетание group/variants, поэтому лучше избавиться от таких данных, чтобы они не искажали результаты теста. 

In [26]:
data.drop(data[(data['group']== 'control') & (data['variants']== 'svm_ensemble_v_1_22')].index, inplace=True)
data.drop(data[(data['group']== 'treatment') & (data['variants']== 'standard')].index,inplace=True)

In [28]:
# Для удобства переименуем группы А - с вариантом сервиса standard и В - с вариантом сервиса  svm_ensemble_v_1_22
data['variants'] = np.where(data['variants']== 'standard', 'A','B')

In [48]:
summary = data.groupby('variants').converted.agg(['sum','mean','count'])
summary

,sum,mean,count
variants,,,
A,17489,0.120386,145274
B,17264,0.118807,145311


In [33]:
(summary['sum'][1]-summary['sum'][0])/summary['sum'][0]*100

-1.286522957287438

In [35]:
(summary['count'][1]-summary['count'][0])/summary['count'][0]*100

0.025469113537178022

**Вывод:**
В эксперименте изначально участвовало 294478 наблюдений с уникальными user-id

Были найдены ошибки в заполнении данных столбцов group/variants, избавились от таких данных, чтобы они не искажали результаты теста. 
Осталось 290585 наблюдений.

145274 пользователей распределенны в группу А и 145311 в группу В.

Относительное количество тестируемых в группе В немного выше чем в группе А на 0,025%

Относительное количество конверсий (бронирований) в группе А немного выше чем в группе В на 1,29%

Среднее значение конверсии для группы А равно 0,120

Среднее значение конверсии для группы В равно 0,119

Проверим, являются ли различия в показателе конверсии ("converted") группы А и группы В статистически значимыми.
Сформулируем наши гипотезы.

**H0 (Null hypothesis)** : Нет статистически значимой разницы в признаке ("converted") в 2-х распределениях

**H1 (Alternative hypothesis)**:  Существует статистически значимая разница в признаке ("converted") в 2-х распределениях

Так как признак у нас бинарный, то мы будем использовать метрику пропорций. Наилучший метод чтобы проверить статистическую значимость изменения для пропорций это метод Хи‑Квадрат. 

Мы посчитаем эту метрику вручную, используя формулу нахождения z-значения, а затем проверим наши расчеты с помощью встроенных методов
sm.stats.proportions_ztest и chi2_contingency, которые должны выдать аналогичный результат, если мы все посчитали правильно.

In [49]:
n1=summary.loc['A', 'count'] #Число успехов выборка А
s1=summary.loc['A', 'sum']  #Число испытаний выборка А
n2=summary.loc['B', 'count'] #Число успехов выборка В
s2=summary.loc['B', 'sum']#Число испытаний выборка В

p1= s1 / n1 # type: ignore
p2= s2 / n2 # type: ignore
p= (s1+s2) / (n1+n2) # type: ignore
z = (p2-p1) / ((p*(1-p)*((1/n1)+(1/n2)))**0.5) # type: ignore
p_value = norm.cdf(z)
print(f'z-значение равно: {round(abs(z),3)},p_value равно: {round(p_value * 2,4)}') # type: ignore

z-значение равно: 1.312,p_value равно: 0.1897


In [50]:
z1, p_value1 = sm.stats.proportions_ztest([s1,s2],[n1,n2])
print(f'z-значение равно: {round(z1,3)},p_value равно: {round(p_value1,4)}') # type: ignore

z-значение равно: 1.312,p_value равно: 0.1897


In [51]:
arr = np.array([[s2,n2-s2],[s1, n1-s1]]) # type: ignore
chi2,p_value3,_,_ = chi2_contingency(arr, correction=False)
print(f'z-значение равно: {round(chi2**0.5,3)},p_value равно: {round(p_value3,4)}')

z-значение равно: 1.312,p_value равно: 0.1897


#### **Вывод:**

- p_value больше 0.05. Таким образом статистический анализ признака converted показал, что статистически значимые результаты отсутствуют в двух вариантах теста и нулевую гипотезу по данному признаку отклонить нельзя.

# **Проведем тест для зарегистрированных пользователей `user_type` = 'registered_user'**

In [60]:
# Сформируем подвыборку, соответствующую условию 'registered_user'
df_reg_user = data[(data['user_type']== 'registered_user')]

In [72]:
summary = df_reg_user.groupby('variants').converted.agg(['sum','count'])
summary

,sum,count
variants,,
A,8689,72483
B,8760,72829


**H0 (Null hypothesis)** : Нет статистически значимой разницы в признаке ("converted") в 2-х распределениях среди зарегистрированных пользователей

**H1 (Alternative hypothesis)**:  Существует статистически значимая разница в признаке ("converted") в 2-х распределениях среди зарегистрированных пользователей

In [76]:
n1=summary.loc['A', 'count'] #Число успехов выборка А
s1=summary.loc['A', 'sum']  #Число испытаний выборка А
n2=summary.loc['B', 'count'] #Число успехов выборка В
s2=summary.loc['B', 'sum']#Число испытаний выборка В
z1, p_value1 = sm.stats.proportions_ztest([s2,s1],[n2,n1])
print(f'z-значение равно: {round(z1,3)},p_value равно: {round(p_value1,4)}') # type: ignore

z-значение равно: 0.238,p_value равно: 0.8121


In [77]:
arr = np.array([[s2,n2-s2],[s1, n1-s1]]) # type: ignore
chi2,p_value3,_,_ = chi2_contingency(arr, correction=False)
print(f'z-значение равно: {round(chi2**0.5,3)},p_value равно: {round(p_value3,4)}')

z-значение равно: 0.238,p_value равно: 0.8121


#### **Вывод:**

- p_value больше 0.05. Таким образом статистический анализ признака converted среди групп зарегистрированных пользователей показал, что статистически значимые результаты отсутствуют в двух вариантах теста и нулевую гипотезу по данному признаку отклонить нельзя.

# **Проведем тест для новых пользователей (new_user**

In [67]:
df_new_user = data[(data['user_type']== 'new_user')]

In [78]:
summary = df_new_user.groupby('variants').converted.agg(['sum','count'])
summary

,sum,count
variants,,
A,8800,72791
B,8504,72482


**H0 (Null hypothesis)** : Нет статистически значимой разницы в признаке ("converted") в 2-х распределениях среди новых пользователей

**H1 (Alternative hypothesis)**:  Существует статистически значимая разница в признаке ("converted") в 2-х распределениях среди новых пользователей

In [83]:
n1=summary.loc['A', 'count'] #Число успехов выборка А
s1=summary.loc['A', 'sum']  #Число испытаний выборка А
n2=summary.loc['B', 'count'] #Число успехов выборка В
s2=summary.loc['B', 'sum']#Число испытаний выборка В
z1, p_value1 = sm.stats.proportions_ztest([s1,s2],[n1,n2])
print(f'z-значение равно: {round(z1,3)},p_value равно: {round(p_value1,4)}')

z-значение равно: 2.099,p_value равно: 0.0358


In [82]:
arr = np.array([[s1, n1-s1], [s2,n2-s2]]) # type: ignore
chi2,p_value3,_,_ = chi2_contingency(arr, correction=False)
print(f'z-значение равно: {round(chi2**0.5,3)},p_value равно: {round(p_value3,4)}')

z-значение равно: 2.099,p_value равно: 0.0358


In [85]:
print(f'Коэф-т converted для А равен: {round(s1/n1,3)}')
print(f'Коэф-т converted для B равен: {round(s2/n2,3)}')

Коэф-т converted для А равен: 0.121
Коэф-т converted для B равен: 0.117


Существует статистически значимая разница в признаке ("converted") в 2-х распределениях среди новых пользователей. В варианте В данный показатель ухудшился и оказываемые изменения для работы сервиса оказывают негативный эффект для новых пользователей. Необходимо избавиться от изменений и вернуться к прежней версии работы сервиса.